# TetCTF 2020 - (Crypto) padwith2019

Challenge's code:

```python
from Crypto.Cipher import AES
import os, json

PAD = bytes.fromhex("2019") * 8


def pad(s):
    pad_length = 16 - len(s) % 16
    return bytes([pad_length]) + PAD[:pad_length - 1] + s


def unpad(s):
    pad_length = s[0]
    if (
            len(s) % 16 != 0 or
            not 1 <= pad_length <= 16 or
            s[1:pad_length] != PAD[:pad_length - 1]
    ):
        raise Exception("incorrect padding")

    return s[pad_length:]


def encrypt(key, plaintext):
    aes = AES.new(key, AES.MODE_CBC, os.urandom(16))
    return aes.IV + aes.encrypt(pad(plaintext))


def decrypt(key, ciphertext):
    aes = AES.new(key, AES.MODE_CBC, ciphertext[:16])
    return unpad(aes.decrypt(ciphertext[16:]))


if __name__ == '__main__':
    key = os.urandom(16)
    obj = {'admin': False, 'flag': open("flag1.txt").read()}
    token = encrypt(key, json.dumps(obj).encode())

    # please decrypt the token!
    print(token.hex())

    for _ in range(65536):
        try:
            token = bytes.fromhex(input())
            obj = json.loads(decrypt(key, token))

            # can you also forge arbitrary tokens?
            if obj['admin']:
                print(open("flag2.txt").read())

        except Exception as e:
            print(e)
```

It's basically a CBC Bit-Flipping Attack challenge (learn more [here](https://masterpessimistaa.wordpress.com/2017/05/03/cbc-bit-flipping-attack/))

In [1]:
from sock import Sock

Firstly, we find padding length by taking advantage of Exception.

In [2]:
s = Sock("207.148.119.58 5555")
token = s.read_line().strip().decode("hex")
iv, ct = token[:16], token[16:]
for i in range(0, 16):
    tmp = list(iv)
    tmp[i] = chr(ord(tmp[i])^1)
    tmp = "".join(tmp)
    aa = tmp + ct
    s.send_line(aa.encode("hex"))
    print s.read_line()
s.close()

incorrect padding

incorrect padding

Expecting value: line 1 column 1 (char 0)

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

'admin'

'admin'

'admin'

'admin'

'admin'

Expecting ':' delimiter: line 1 column 19 (char 18)

Expecting ':' delimiter: line 1 column 9 (char 8)

Expecting value: line 1 column 10 (char 9)

Expecting value: line 1 column 11 (char 10)

Expecting value: line 1 column 11 (char 10)

Expecting value: line 1 column 11 (char 10)

Expecting value: line 1 column 11 (char 10)



So length of padding is 2, and plaintext has form `xx{"admin": fals|e, "flag": "TetC|TF{aaaaaaaaaaa"}`.
Now we will flip `fals` to ` true`

In [3]:
s = Sock("207.148.119.58 5555")
token = s.read_line().strip().decode("hex")
iv, ct = token[:16], token[16:]
aaa = list(iv)
old = "fals"
new = " tru"
for i in range(4):
    aaa[12+i] = chr(ord(aaa[12+i]) ^ ord(old[i]) ^ ord(new[i]))
    bbb = "".join(aaa) + ct
s.send_line(bbb.encode("hex"))
part2 = s.read_line()
s.close()
print part2

th3_b3g1nn1ng_d03s_n0t_h3lp}



We got the second part of the flag, now we'll use the Exception again to recover the first part.

In [4]:
s = Sock("207.148.119.58 5555")
token = s.read_line().strip().decode("hex")
iv, ct = token[:16], token[16:]
import string
PAD = "\x20\x19" * 8
iv, ct = ct[16:32], ct[32:]
part1 = "TF{"
sig = string.ascii_letters + string.digits + "}{_"
for i in range(3, 16, 1):
    aaa = list(iv)
    aaa[0] = chr(ord(aaa[0]) ^ (i+1) ^ ord("T"))
    for j in range(1, i, 1):
        aaa[j] = chr(ord(aaa[j]) ^ ord(part1[j]) ^ ord(PAD[j-1]))
    tmp = aaa[i]
    for j in sig:
        aaa[i] = chr(ord(tmp) ^ ord(j) ^ (ord(PAD[i-1])))
        bbb = "".join(aaa) + ct
        s.send_line(bbb.encode("hex"))
        if "padding" not in s.read_line():
            part1 += j
            print part1
            break
s.close()

TF{p
TF{p4
TF{p4d
TF{p4dd
TF{p4dd1
TF{p4dd1n
TF{p4dd1ng
TF{p4dd1ng_
TF{p4dd1ng_4
TF{p4dd1ng_4t
TF{p4dd1ng_4t_


IndexError: string index out of range

Done! The flag is `TetCTF{p4dd1ng_4t_th3_b3g1nn1ng_d03s_n0t_h3lp}`